# Moon

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@feature-in-context-learning 

In [2]:
#%%capture
#!pip install git+https://github.com/fuyu-quant/IBLM.git

In [3]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.45


In [4]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

import sys
import os

In [5]:
n = 400

df = pd.read_csv(f'../data/moon/moon_{n}_train.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']
print(len(x_train))

400


In [6]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

### Training

In [7]:
file_path = '../models/moon/'

model = iblm.fit(x_train, y_train, model_name = 'moon', file_path=file_path)

Tokens Used: 5441
	Prompt Tokens: 5258
	Completion Tokens: 183
Successful Requests: 1
Total Cost (USD): $0.16871999999999998


In [9]:
# Code of the model created
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        
        # Calculate the distance from the center of each cluster
        distance_1 = np.sqrt((row['Feature_1'] - 1)**2 + (row['Feature_2'] - 0)**2)
        distance_0 = np.sqrt((row['Feature_1'] - 0)**2 + (row['Feature_2'] - 1)**2)
        
        # Calculate the probability of belonging to cluster 1 (target = 1)
        prob_1 = 1 / (1 + np.exp(distance_1 - distance_0))
        
        # Do not change the code after this point.
        output.append(prob_1)
    return np.array(output)


### Prediction

In [10]:
df = pd.read_csv(f'../data/moon/moon_{n}_test.csv')
x_test = df.drop('Target', axis=1)
y_test = df['Target']

In [11]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [12]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.8066666666666666
Precision: 0.7787878787878788
Recall: 0.8566666666666667
F1 score: 0.8158730158730159
ROC-AUC: 0.8890555555555555


## Create multiple code models

In [13]:
n = 600
df = pd.read_csv(f'../data/moon/moon_{n}_train.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']

file_path = '../models/moon/'
for i in range(1,31):
    model = iblm.fit(x_train, y_train, model_name = f'moon_{i}', file_path=file_path)

Tokens Used: 4716
	Prompt Tokens: 4575
	Completion Tokens: 141
Successful Requests: 1
Total Cost (USD): $0.14571
Tokens Used: 4760
	Prompt Tokens: 4575
	Completion Tokens: 185
Successful Requests: 1
Total Cost (USD): $0.14835
Tokens Used: 4716
	Prompt Tokens: 4575
	Completion Tokens: 141
Successful Requests: 1
Total Cost (USD): $0.14571
Tokens Used: 4713
	Prompt Tokens: 4575
	Completion Tokens: 138
Successful Requests: 1
Total Cost (USD): $0.14553000000000002
Tokens Used: 4718
	Prompt Tokens: 4575
	Completion Tokens: 143
Successful Requests: 1
Total Cost (USD): $0.14583000000000002
Tokens Used: 4718
	Prompt Tokens: 4575
	Completion Tokens: 143
Successful Requests: 1
Total Cost (USD): $0.14583000000000002
Tokens Used: 4718
	Prompt Tokens: 4575
	Completion Tokens: 143
Successful Requests: 1
Total Cost (USD): $0.14583000000000002
Tokens Used: 4760
	Prompt Tokens: 4575
	Completion Tokens: 185
Successful Requests: 1
Total Cost (USD): $0.14835
Tokens Used: 4718
	Prompt Tokens: 4575
	Completi

### Prediction

In [20]:
n = 400

df = pd.read_csv(f'../data/moon/moon_{n}_test.csv')
x_test = df.drop('Target', axis=1)
y_test = df['Target']

sys.path.append('..')

error_count = 0
error_list = []
auc_list = []
acc_list = []

for i in range(1,31):

    import_file = f'import models.moon.moon_{i} as codemodel'

    exec(import_file)

    try:
        y_proba = codemodel.predict(x_test)
        y_pred = (y_proba > 0.5).astype(int)
        negative_values_exist = np.any(y_proba < 0)
        values_greater_than_one_exist = np.any(y_proba > 1)
        if negative_values_exist:
            error_list.append(i)
            error_count += 1
            print(f"Negative values exist：{negative_values_exist}")

        elif values_greater_than_one_exist:
            error_list.append(i)
            error_count += 1
            print(f"Positive values exist：{values_greater_than_one_exist}")

        else:
            roc_auc = roc_auc_score(y_test, y_proba)
            accuracy = round(accuracy_score(y_test, y_pred),4)
            auc_list.append(roc_auc)
            acc_list.append(accuracy)
        
    except Exception:
        print('run error')
        error_count += 1
        error_list.append(i)
        pass

print('-----------error-------------')
print(error_count)
print(error_list)

print('-----------AUC-------------')
print(auc_list)
average = sum(auc_list) / len(auc_list)
print("Average Value:", average)
max_value = max(auc_list)
min_value = min(auc_list)
print("Maximum Value:", max_value)
print("Minimum Value:", min_value)

print('-----------ACC-------------')
print(acc_list)
average = sum(acc_list) / len(acc_list)
print("Average Value:", average)
max_value = max(acc_list)
min_value = min(acc_list)
print("Maximum Value:", max_value)
print("Minimum Value:", min_value)

-----------error-------------
0
[]
-----------AUC-------------
[0.511, 0.8890555555555555, 0.511, 0.511, 0.6252333333333333, 0.6252333333333333, 0.6252333333333333, 0.8890555555555555, 0.6252333333333333, 0.53015, 0.511, 0.5, 0.6252333333333333, 0.6252333333333333, 0.511, 0.511, 0.53015, 0.53015, 0.8890555555555555, 0.5, 0.53015, 0.6252333333333333, 0.53015, 0.5, 0.6252333333333333, 0.8890555555555555, 0.5, 0.53015, 0.6252333333333333, 0.5301500000000001]
Average Value: 0.5986790740740742
Maximum Value: 0.8890555555555555
Minimum Value: 0.5
-----------ACC-------------
[0.5, 0.8067, 0.5, 0.5, 0.6483, 0.6483, 0.6483, 0.8067, 0.6483, 0.6167, 0.5, 0.5, 0.6483, 0.6483, 0.5, 0.5, 0.6167, 0.6167, 0.8067, 0.5, 0.6167, 0.6483, 0.6167, 0.5, 0.6483, 0.8067, 0.5, 0.6167, 0.6483, 0.6167]
Average Value: 0.6126133333333333
Maximum Value: 0.8067
Minimum Value: 0.5


In [18]:
n = 400
df = pd.read_csv(f'../data/moon/moon_{n}_test.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']

import models.moon.moon_2 as codemodel

y_proba = codemodel.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.8066666666666666
Precision: 0.7787878787878788
Recall: 0.8566666666666667
F1 score: 0.8158730158730159
ROC-AUC: 0.8890555555555555
